In [2]:
%load_ext autoreload
%autoreload 1

In [3]:
%%file GRBL.py
import serial
class GRBL(object):
    BAUDRATE = 115200
    
    def __init__(self, port):
        self.serial = serial.Serial(port=port,
                                    baudrate=GRBL.BAUDRATE,
                                    timeout=0.10)
        
    def write(self, command_line=""):
        self.serial.flushInput()
        self.serial.write("\n".encode())
        self.serial.write("{cmd}\n".format(cmd=command_line).encode())
        
    def read(self, multiline=True):
        if multiline:
            responses = self.serial.readlines()
            responses = [response.decode().strip() for response in responses]
            return responses
        else:
            response = self.serial.readline()
            return response.decode().strip()
 
    def cmd(self, command_line, resp=True, multiline=True):
        self.write(command_line)
        if resp:
            return self.read(multiline=multiline)
        return None
    
    def reset(self):
        ret = self.cmd("\x18")
        assert(ret[-1]=='ok')
        
    def sleep(self):
        ret = self.cmd("\x18")
        assert(ret[-1]=='ok')
    
    def kill_alarm(self):
        ret = self.cmd("$X")
        assert(ret[-1]=='ok')
        
    def home(self):
        self.write("$H")
        

settings = [
    ("$0", "step_pulse"),
    ("$1", "step_idle_delay"),
    ("$2", "step_port_invert"),
    ("$3", "direction_port_invert"),
    ("$4", "step_enable_invert"),
    ("$5", "limit_pin_invert"),
    ("$6", "probe_pin_invert"),
    ("$10", "status_report"),
    ("$11", "junction_deviation"),
    ("$12", "arc_tolerance"),
    ("$13", "report_inches"),
    ("$20", "soft_limits"),
    ("$21", "hard_limits"),
    ("$22", "homing_cycle"),
    ("$23", "homing_dir_invert"),
    ("$24", "homing_feed"),
    ("$25", "homing_seek"),
    ("$26", "homing_debounce"),
    ("$27", "homing_pull_off"),
    ("$30", "max_spindle_speed"),
    ("$31", "min_spindle_speed"),
    ("$32", "laser_mode"),
    ("$100", "x_steps_mm"),
    ("$101", "y_steps_mm"),
    ("$102", "z_steps_mm"),
    ("$110", "x_max_rate"),
    ("$111", "y_max_rate"),
    ("$112", "z_max_rate"),
    ("$120", "x_acceleration"),
    ("$121", "y_acceleration"),
    ("$122", "z_acceleration"),
    ("$130", "x_travel"),
    ("$131", "y_travel"),
    ("$132", "z_travel"),
    ]

def grbl_getter_generator(cmd):
    def grbl_getter(self):
        config = self.cmd("$$", resp=True, multiline=True)
        for config_line in config:
            if config_line.startswith("$"):
                key, value = config_line.split("=")
                if key == cmd:
                    return float(value)
        return None
    return grbl_getter
    
def grbl_setter_generator(cmd):
    def grbl_setter(self, value):
        set_cmd = "{cmd}={value}".format(cmd=cmd, value=value)
        ret = self.cmd(set_cmd, resp=True, multiline=False)
        print(ret)
        
    return grbl_setter

for setting in settings:
    cmd = setting[0]
    name = setting[1]
    
    setter = grbl_setter_generator(cmd)
    getter = grbl_getter_generator(cmd)
    
    prop = property(fget=getter,
                    fset=setter,
                    doc=" ".join(name.split("_")))
    
    setattr(GRBL, name, prop)

Overwriting GRBL.py


In [4]:
%aimport GRBL

In [5]:
grbl = GRBL.GRBL("/dev/cnc_3018")

# Grbl '$' Commands

In [6]:
grbl.cmd("?")

['ok', '<Idle|MPos:0.000,0.000,0.000|FS:0,0|WCO:0.000,0.000,0.000>', 'ok']

In [7]:
grbl.cmd("$?")

['ok',
 '<Idle|MPos:0.000,0.000,0.000|FS:0,0|Ov:100,100,100>',
 '[HLP:$$ $# $G $I $N $x=val $Nx=line $J=line $SLP $C $X $H ~ ! ? ctrl-x]',
 'ok']

In [11]:
gcode_parameters = grbl.cmd("$#") # View gcode parameters
gcode_parameters

['ok',
 '[G54:0.000,0.000,0.000]',
 '[G55:0.000,0.000,0.000]',
 '[G56:0.000,0.000,0.000]',
 '[G57:0.000,0.000,0.000]',
 '[G58:0.000,0.000,0.000]',
 '[G59:0.000,0.000,0.000]',
 '[G28:0.000,0.000,0.000]',
 '[G30:0.000,0.000,0.000]',
 '[G92:0.000,0.000,0.000]',
 '[TLO:0.000]',
 '[PRB:0.000,0.000,0.000:0]',
 'ok']

In [44]:
def gcode_param_gen(parameter):
    def gcode_param(self):
        gcode_parameters = self.cmd("$#") # View gcode parameters
        for gcode_parameter in gcode_parameters: 
            if parameter in gcode_parameter:
                _, value = gcode_parameter.split(":")
                value = value.strip("]")
                values = value.split(",")
                values = [float(value) for value in values]
                
                return values
        return None
    return gcode_param

G54 = gcode_param_gen("G54")

G54(grbl)

[0.0, 0.0, 0.0]

In [51]:
print("gcode_parameters = [")
for p in gcode_parameters:
    if p.startswith("["):
        k, *_ = p.split(":")
        k = k.strip("[")
        print("    \"{}\",".format(k))
print("    ]")

gcode_parameters = [
    "G54",
    "G55",
    "G56",
    "G57",
    "G58",
    "G59",
    "G28",
    "G30",
    "G92",
    "TLO",
    "PRB",
    ]


In [52]:
gcode_parameters = [
    "G54",
    "G55",
    "G56",
    "G57",
    "G58",
    "G59",
    "G28",
    "G30",
    "G92",
    "TLO",
    "PRB",
    ]

In [55]:
for parameter in gcode_parameters:
    fcn = gcode_param_gen(parameter)
    prop = property(fget=fcn)
    setattr(GRBL.GRBL, parameter, prop)

In [56]:
grbl = GRBL.GRBL("/dev/cnc_3018")

In [57]:
grbl.G54

[0.0, 0.0, 0.0]

In [ ]:
def status(self):
    ret = self.cmd("")